In [1]:
!sudo apt-get update -y
!apt-get install -y libvulkan-dev  glslang-tools libxxf86vm-dev libxrandr-dev libwayland-dev libnvidia-ml-dev g++ nvidia-cuda-toolkit

!sudo apt-get install -y --no-install-recommends \
    vulkan-tools \
    libvulkan-dev \
    nvidia-driver-525 \
    nvidia-utils-525

# 2. Create symbolic links for Vulkan loader
!sudo ln -s /usr/lib/x86_64-linux-gnu/libvulkan.so.1.3.224 /usr/lib/x86_64-linux-gnu/libvulkan.so.1
!sudo ln -s /usr/lib/x86_64-linux-gnu/libvulkan.so.1 /usr/lib/x86_64-linux-gnu/libvulkan.so



# 3. Set environment variables
import os
os.environ['VK_ICD_FILENAMES'] = '/usr/share/vulkan/icd.d/nvidia_icd.json'



Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [1,804 kB]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:6 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:7 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:8 https://r2u.stat.illinois.edu/ubuntu jammy/main all Packages [9,077 kB]
Hit:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease [24.3 kB]
Get:11 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:12 http://security.ubuntu.com/ubuntu jammy-security/universe amd64 Packages [1,262 kB]
Hit:13 https://ppa.launchp

In [2]:
# 2. Install only essential Vulkan components
!sudo apt-get update && sudo apt-get install -y --no-install-recommends \
    libvulkan1 \
    vulkan-utils \
    vulkan-tools \
    libvulkan-dev \
    nvidia-utils-535-server \
    nvidia-opencl-icd-535-server

# 3. Configure symbolic links for Vulkan
!sudo ln -sf /usr/lib/x86_64-linux-gnu/libvulkan.so.1.3.224 /usr/lib/x86_64-linux-gnu/libvulkan.so.1
!sudo ln -sf /usr/lib/x86_64-linux-gnu/libvulkan.so.1 /usr/lib/x86_64-linux-gnu/libvulkan.so
!sudo ldconfig

# 4. Verify installation
!vulkaninfo --summary
!nvidia-smi

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:5 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:7 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading package lists... Done
Building dependency tree... Done
Reading

In [3]:
# Install required packages
!apt-get update
!apt-get install -y cmake git build-essential python3

# Clone glslang with its dependencies
!git clone https://github.com/KhronosGroup/glslang.git
%cd glslang
!python3 /content/glslang/update_glslang_sources.py

# Configure and build
!cmake -S . -B build -DCMAKE_BUILD_TYPE=Release -DENABLE_GLSLANG_BINARIES=ON -DBUILD_SHARED_LIBS=OFF -DENABLE_OPT=OFF
!cmake --build build -j4

# Copy the binary to a global path
!cp build/StandAlone/glslangValidator /usr/local/bin/


Hit:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:4 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:7 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading package lists... Done
Building dependency tree... Done
Reading

In [1]:
!/content/glslang/build/StandAlone/glslangValidator --version


Glslang Version: 11:15.4.0
ESSL Version: OpenGL ES GLSL 3.20 glslang Khronos. 15.4.0
GLSL Version: 4.60 glslang Khronos. 15.4.0
SPIR-V Version 0x00010600, Revision 1
GLSL.std.450 Version 100, Revision 1
Khronos Tool ID 8
SPIR-V Generator Version 11
GL_KHR_vulkan_glsl version 100
ARB_GL_gl_spirv version 100


In [44]:
%%writefile compute_shader.comp
#version 450
layout(local_size_x = 1024) in;
layout(binding = 0) buffer Buf { float data[]; } buf;
void main() {
    uint gid = gl_GlobalInvocationID.x;
    float x = buf.data[gid];
    for (int i = 0; i < 10000000; i++) {
        x = fract(tan(x * i + gl_GlobalInvocationID.x) * 1e9);
        buf.data[gid] = x;
        x = pow(x, x);
        memoryBarrier();
        x = length(vec2(x, fract(x * 0.1)));
    }
    buf.data[gid] = x;
}

Overwriting compute_shader.comp


In [26]:
%%writefile cuda_stress.cu
#include <cuda_runtime.h>
#include <cstdio>
#include <cstdint>


__device__ uint8_t* g_bigBuf = nullptr;
__device__ size_t   g_numBytes = 0;

__global__ void memFlood()
{
    uint64_t* p64   = reinterpret_cast<uint64_t*>(g_bigBuf);
    size_t    words = g_numBytes / sizeof(uint64_t);
    size_t    tid   = blockIdx.x * blockDim.x + threadIdx.x;
    size_t    step  = gridDim.x * blockDim.x;

    for (size_t i = tid; i < words; i += step)
        p64[i] = p64[i] * 0x9E3779B97F4A7C15ull + 0x5A5A5A5A5A5A5A5Aull;
}

extern "C" void launchHeavyKernel()
{
    static bool    first = true;
    static size_t  bytes = 0;
    static uint8_t* dptr = nullptr;

    if (first) {
        size_t freeB, totalB;
        cudaMemGetInfo(&freeB, &totalB);          // free before we allocate
        bytes = size_t(freeB * 0.80);             // ~80 % of what’s left
        bytes &= ~(size_t(256 * 1024 * 1024) - 1); // round down to 256 MB
        cudaMalloc(&dptr, bytes);
        cudaMemcpyToSymbol(g_bigBuf,   &dptr,  sizeof(uint8_t*));
        cudaMemcpyToSymbol(g_numBytes, &bytes, sizeof(size_t));
        printf("Allocated %.2f GB flood buffer\n", bytes / 1e9);
        first = false;
    }

    dim3 blk(1024);
    dim3 grd(65535);
    while (true) {
        memFlood<<<grd, blk>>>();
        cudaDeviceSynchronize();
    }
}

Overwriting cuda_stress.cu


In [50]:
%%writefile main.cpp
#include <iostream>
#include <thread>
#include <vector>
#include <atomic>
#include <random>
#include <chrono>
#include <pthread.h>
#include <sched.h>
#include </usr/local/cuda/include/nvml.h>
#include <vulkan/vulkan.hpp>
#include </usr/local/cuda/include/cuda_runtime.h>
#include <fstream>
#include <iomanip>

#define CUDA_CHECK(call) \
    do { cudaError_t err = call; \
        if (err != cudaSuccess) { \
            std::cerr << "CUDA error: " << cudaGetErrorString(err) \
                      << " at " << __FILE__ << ":" << __LINE__ << "\n"; \
            exit(EXIT_FAILURE); \
        } \
    } while(0)

extern "C" void launchHeavyKernel();

// Function to load SPIR-V binary
std::vector<uint32_t> loadSPIRV(const char* filename) {
    std::ifstream file(filename, std::ios::binary);
    if (!file) {
        throw std::runtime_error("Failed to open SPIR-V file");
    }

    file.seekg(0, std::ios::end);
    size_t size = file.tellg();
    file.seekg(0, std::ios::beg);

    if (size % sizeof(uint32_t) != 0) {
        throw std::runtime_error("Invalid SPIR-V file size");
    }

    std::vector<uint32_t> spirv(size / sizeof(uint32_t));
    file.read(reinterpret_cast<char*>(spirv.data()), size);
    return spirv;
}

constexpr size_t VK_BUFFER_SIZE = 20 * 1024 * 1024;  // 200MB buffer

void runVulkanStress(std::atomic<bool>& stopFlag) {
    try {
        vk::Instance instance = vk::createInstance({});
        vk::PhysicalDevice physicalDevice = instance.enumeratePhysicalDevices().front();

        float queuePriority = 1.0f;
        vk::DeviceQueueCreateInfo queueInfo({}, 0, 1, &queuePriority);
        vk::DeviceCreateInfo deviceCreateInfo({}, 1, &queueInfo);
        vk::Device device = physicalDevice.createDevice(deviceCreateInfo);

        vk::BufferCreateInfo bufferInfo({}, VK_BUFFER_SIZE,
            vk::BufferUsageFlagBits::eStorageBuffer, vk::SharingMode::eExclusive);
        vk::Buffer buffer = device.createBuffer(bufferInfo);

        vk::MemoryRequirements memReq = device.getBufferMemoryRequirements(buffer);

        // Vulkan memory allocation
        vk::PhysicalDeviceMemoryProperties memProps = physicalDevice.getMemoryProperties();
        uint32_t memoryTypeIndex = uint32_t(~0);
        for (uint32_t i = 0; i < memProps.memoryTypeCount; ++i) {
            if ((memReq.memoryTypeBits & (1 << i)) &&
                  (memProps.memoryTypes[i].propertyFlags & vk::MemoryPropertyFlagBits::eDeviceLocal)) {
                memoryTypeIndex = i;
                break;
            }
        }

        if (memoryTypeIndex == uint32_t(~0)) {
            throw std::runtime_error("No suitable memory type found!");
        }

        vk::MemoryAllocateInfo allocInfo(memReq.size, memoryTypeIndex);
        vk::DeviceMemory memory = device.allocateMemory(allocInfo);
        device.bindBufferMemory(buffer, memory, 0);

        auto spirvCode = loadSPIRV("heavy_transformer_block.spv");


        vk::ShaderModule shaderModule = device.createShaderModule(
            {{}, spirvCode.size() * sizeof(uint32_t), spirvCode.data()});

        vk::PipelineLayout pipelineLayout = device.createPipelineLayout({});
        vk::PipelineCache pipelineCache = device.createPipelineCache({});
        vk::ComputePipelineCreateInfo pipelineInfo({},
            {{}, vk::ShaderStageFlagBits::eCompute, shaderModule, "main"},
            pipelineLayout);
        vk::Pipeline pipeline = device.createComputePipeline(pipelineCache, pipelineInfo).value;

        vk::CommandPool commandPool = device.createCommandPool({});
        vk::CommandBuffer cmdBuf = device.allocateCommandBuffers(
            {commandPool, vk::CommandBufferLevel::ePrimary, 1})[0];

        cmdBuf.begin({vk::CommandBufferUsageFlagBits::eSimultaneousUse});
        cmdBuf.bindPipeline(vk::PipelineBindPoint::eCompute, pipeline);
        cmdBuf.dispatch(65535, 65535, 1);
        cmdBuf.end();

        // Create multiple queues
        std::vector<vk::Queue> queues(4, device.getQueue(0, 0));
        std::vector<vk::Fence> fences(4);
        for (auto& fence : fences) fence = device.createFence({});

        while (!stopFlag.load()) {
            for (int i = 0; i < 4; i++) {
                queues[i].submit({vk::SubmitInfo(0, nullptr, nullptr, 1, &cmdBuf)}, fences[i]);
            }
            for (auto& fence : fences) {
                (void)device.waitForFences(fence, true, UINT64_MAX);
                device.resetFences(fence);
            }
        }

        device.destroy();
        instance.destroy();
    } catch (vk::Error& err) {
        std::cerr << "Vulkan error: " << err.what() << std::endl;
        exit(EXIT_FAILURE);
    }
}

void cpuStressWorker(int coreId, std::atomic<bool>& stopFlag) {
    cpu_set_t cpuset;
    CPU_ZERO(&cpuset);
    CPU_SET(coreId, &cpuset);
    pthread_setaffinity_np(pthread_self(), sizeof(cpu_set_t), &cpuset);

        const size_t arrSize = 800'000'000;
    alignas(64) volatile float* data = new float[arrSize];

    while (!stopFlag.load()) {
        #pragma omp simd  // Force vectorization
        for (size_t i = 0; i < arrSize; ++i) {
            data[i] = sinf(data[i]) * coshf(data[i]) + sqrtf(fabsf(data[i]));
        }
    }
    delete[] data;
}

void pcieStress(std::atomic<bool>& stopFlag) {
    CUDA_CHECK(cudaSetDevice(0));
    const size_t bytes = 2ULL * 1024 * 1024 * 1024;  // 2GB
    const int numStreams = 16;
    cudaStream_t streams[numStreams];
    void *h_ptr, *d_ptr;

    CUDA_CHECK(cudaHostAlloc(&h_ptr, bytes, cudaHostAllocDefault));
    CUDA_CHECK(cudaMalloc(&d_ptr, bytes));
    for (int i = 0; i < numStreams; i++) {
        CUDA_CHECK(cudaStreamCreate(&streams[i]));
    }

    while (!stopFlag.load()) {
        for (int i = 0; i < numStreams; i++) {
            cudaMemcpyAsync(d_ptr, h_ptr, bytes/numStreams,
                           cudaMemcpyHostToDevice, streams[i]);
            cudaMemcpyAsync(h_ptr, d_ptr, bytes/numStreams,
                           cudaMemcpyDeviceToHost, streams[i]);
        }
        cudaDeviceSynchronize();
    }
    cudaFree(d_ptr);
    cudaFreeHost(h_ptr);
}

void nvmlLogging(std::atomic<bool>& stopFlag)
{
    nvmlInit();
    nvmlDevice_t dev;
    nvmlDeviceGetHandleByIndex(0, &dev);

    while (!stopFlag.load())
    {
        // ① bandwidth-style utilisation (what you had)
        nvmlUtilization_t util {};
        nvmlDeviceGetUtilizationRates(dev, &util);

        // ② actual frame-buffer occupancy
        nvmlMemory_t mem {};
        nvmlDeviceGetMemoryInfo(dev, &mem);
        double usedGiB  = mem.used  / (1024.0 * 1024 * 1024);
        double totalGiB = mem.total / (1024.0 * 1024 * 1024);
        double pctVRAM  = 100.0 * mem.used / mem.total;

        // ③ temperature just like before
        unsigned int temp = 0;
        nvmlDeviceGetTemperature(dev, NVML_TEMPERATURE_GPU, &temp);

        std::cout << "[NVML] GPU: "
                  << util.gpu    << "%  |  MemCtl: "
                  << util.memory << "%  |  VRAM: "
                  << std::fixed << std::setprecision(2)
                  << usedGiB << '/' << totalGiB << " GiB ("
                  << std::setprecision(1) << pctVRAM << "%)  |  Temp: "
                  << temp << " °C\n";

        std::this_thread::sleep_for(std::chrono::milliseconds(500));
    }
    nvmlShutdown();
}


int main() {

    std::atomic<bool> stopFlag(false);
    std::vector<std::thread> threads;

    // CUDA Stress (4000 blocks x 512 threads)
    threads.emplace_back([&stopFlag](){
        CUDA_CHECK(cudaSetDevice(0));
        while (!stopFlag.load()) {
          std :: cout << "Launching Heavy Kernel"<<std ::endl;
            launchHeavyKernel();
        }
    });

    // Vulkan Compute
    threads.emplace_back(runVulkanStress, std::ref(stopFlag));

     // CPU Stress (All cores)
    int numCores = std::thread::hardware_concurrency();
    for (int i = 0; i < numCores; ++i) {
         threads.emplace_back(cpuStressWorker, i, std::ref(stopFlag));
     }

    // PCIe Stress
    threads.emplace_back(pcieStress, std::ref(stopFlag));

    // Monitoring
    threads.emplace_back(nvmlLogging, std::ref(stopFlag));

    std::this_thread::sleep_for(std::chrono::seconds(300));
    stopFlag.store(true);

    for (auto& t : threads) if (t.joinable()) t.join();
    return 0;
}

Overwriting main.cpp


In [43]:
%cd /content/glslang

# ➊ GLSL → SPIR-V
!/content/glslang/build/StandAlone/glslangValidator -V compute_shader.comp -o compute_shader.spv \
                 --target-env vulkan1.1 -e main -S comp

# ➋ CUDA object
!nvcc -std=c++17 -O3 -c cuda_stress.cu -o cuda_stress.o

# ➌ Link everything (nvcc links CUDA libs for you)
!g++ -std=c++17 -O3 main.cpp cuda_stress.o -o nuclear_test -L/usr/local/cuda/lib64 -lnvidia-ml -lvulkan -lpthread -lcudart -fopenmp


/content/glslang
compute_shader.comp
ERROR: compute_shader.comp:53: '' :  syntax error, unexpected STAR, expecting RIGHT_PAREN
ERROR: 1 compilation errors.  No code generated.


ERROR: Linking compute stage: Missing entry point: Each stage requires one entry point

SPIR-V is not generated for failed compile or link
^C


In [45]:
!./nuclear_test

[NVML] GPU: 0%  |  MemCtl: 0%  |  VRAM: 0.26/15.00 GiB (1.7%)  |  Temp: 48 °C
^C


In [48]:
%%writefile heavy_transformer_block.comp
#version 450
/* ─────────────── Compile with: ───────────────
   glslangValidator -V heavy_transformer_block.comp \
                    -o heavy_transformer_block.spv \
                    --target-env vulkan1.1 -e main -S comp
   ──────────────────────────────────────────── */

layout(local_size_x = 32, local_size_y = 8, local_size_z = 1) in;
/* 256 threads/WG → good occupancy */

const uint D   = 8192u;   /* hidden size (embeddings) */
const uint H   = 32u;     /* attention heads          */
const uint Dh  = D / H;   /* 256 per head             */
const uint FF  = D * 4u;  /* feed-forward expansion   */
const uint B   = 128u;    /* batch-seq tokens         */

layout(binding =  0) readonly  buffer QKBuf { float Wqk[D*D]; };   /*  256 MB */
layout(binding =  1) readonly  buffer VBuf  { float Wv[D*D];  };   /*  256 MB */
layout(binding =  2) readonly  buffer OBuf  { float Wo[D*D];  };   /*  256 MB */
layout(binding =  3) readonly  buffer F1Buf { float Wf1[D*FF]; };  /* 1024 MB */
layout(binding =  4) readonly  buffer F2Buf { float Wf2[FF*D]; };  /* 1024 MB */
layout(binding =  5) readonly  buffer InBuf { float  X[B*D];   };  /*   4 MB  */
layout(binding =  6) writeonly buffer OutBuf{ float  Y[B*D];   };

shared float Asub[32][32];  /*  4 KB */
shared float Bsub[32][32];  /*  4 KB */

uint idx(uint r, uint c, uint ld) { return r*ld + c; }

/* === tiny FP32 GELU === */
float gelu(float x){
    const float k0 = 0.7978845608;
    const float k1 = 0.044715;
    return 0.5*x*(1.0 + tanh(k0*(x + k1*x*x*x)));
}

/* === 32×32 tiled GEMM kernel (one output row) === */
void gemmTile(in uint baseA, in uint lda,
              in uint baseB, in uint ldb,
              in uint K,
              inout float acc[32])
{
    for(uint k0=0u;k0<K;k0+=32u){
        /* load tiles */
        Asub[gl_LocalInvocationID.y][gl_LocalInvocationID.x] =
            Wqk[baseA + idx(gl_WorkGroupID.y*32u+gl_LocalInvocationID.y,
                            k0+gl_LocalInvocationID.x, lda)];
        Bsub[gl_LocalInvocationID.y][gl_LocalInvocationID.x] =
            X   [baseB + idx(k0+gl_LocalInvocationID.y,
                            gl_WorkGroupID.x*32u+gl_LocalInvocationID.x, ldb)];
        memoryBarrierShared(); barrier();

        /* FMA loop */
        for(uint k=0u;k<32u;++k)
            acc[gl_LocalInvocationID.x] +=
                Asub[gl_LocalInvocationID.y][k] *
                Bsub[k][gl_LocalInvocationID.x];
        barrier();
    }
}

void main(){
    /* Output tile indices */
    uint row = gl_WorkGroupID.y*32u + gl_LocalInvocationID.y;  /* token */
    uint col = gl_WorkGroupID.x*32u + gl_LocalInvocationID.x;  /* hidden */
    if(row>=B||col>=D) return;

    /* ---------- Attention QKᵀ softmax V skipped (bandwidth killer) ---------- */
    /* For stress-only purposes we replace attention with a dense GEMM ……     */
    float acc[32]; for(int i=0;i<32;i++) acc[i]=0.0;
    gemmTile(0u, D, 0u, D, D, acc);          /* Wqk × X */

    /* Apply GELU and write temp into shared mem for the Wo projection */
    Asub[gl_LocalInvocationID.y][gl_LocalInvocationID.x] =
        gelu(acc[gl_LocalInvocationID.x]);
    barrier();

    /* ── Project back (Wo) ── */
    float proj = 0.0;
    for(uint k0=0u;k0<D;k0+=32u){
        Bsub[gl_LocalInvocationID.y][gl_LocalInvocationID.x] =
            Wo[idx(k0+gl_LocalInvocationID.y,col,D)];
        memoryBarrierShared(); barrier();
        for(uint k=0u;k<32u;++k)
            proj += Asub[gl_LocalInvocationID.y][k]*Bsub[k][gl_LocalInvocationID.x];
        barrier();
    }

    /* ---------- Feed-forward  (4×D → D) ---------- */
    float ff1 = 0.0;
    for(uint k0=0u;k0<FF;k0+=32u){
        /* Wf1 is D×FF  (big!) */
        Bsub[gl_LocalInvocationID.y][gl_LocalInvocationID.x] =
            Wf1[idx(row,k0+gl_LocalInvocationID.x,FF)];
        memoryBarrierShared(); barrier();
        for(uint k=0u;k<32u;++k)
            ff1 += Bsub[gl_LocalInvocationID.y][k]*proj;   /* broadcast proj */
        barrier();
    }
    ff1 = gelu(ff1);

    float ff2 = 0.0;
    for(uint k0=0u;k0<FF;k0+=32u){
        Bsub[gl_LocalInvocationID.y][gl_LocalInvocationID.x] =
            Wf2[idx(k0+gl_LocalInvocationID.y,col,D)];
        memoryBarrierShared(); barrier();
        for(uint k=0u;k<32u;++k)
            ff2 += ff1*Bsub[k][gl_LocalInvocationID.x];
        barrier();
    }

    /* Output */
    Y[idx(row,col,D)] = ff2;
}


Writing heavy_transformer_block.comp


In [51]:

!/content/glslang/build/StandAlone/glslangValidator -V heavy_transformer_block.comp \
                 -o heavy_transformer_block.spv \
                 --target-env vulkan1.1 -e main -S comp

# ➋ CUDA object
!nvcc -std=c++17 -O3 -c cuda_stress.cu -o cuda_stress.o

# ➌ Link everything (nvcc links CUDA libs for you)
!g++ -std=c++17 -O3 main.cpp cuda_stress.o -o nuclear_test -L/usr/local/cuda/lib64 -lnvidia-ml -lvulkan -lpthread -lcudart -fopenmp



heavy_transformer_block.comp


In [52]:
!./nuclear_test

[NVML] GPU: 0%  |  MemCtl: 0%  |  VRAM: 0.26/15.00 GiB (1.7%)  |  Temp: 46 °C
Launching Heavy Kernel
[NVML] GPU: 2%  |  MemCtl: 0%  |  VRAM: 11.86/15.00 GiB (79.1%)  |  Temp: 46 °C
[NVML] GPU: 0%  |  MemCtl: 0%  |  VRAM: 11.86/15.00 GiB (79.1%)  |  Temp: 47 °C
Allocated 12.35 GB flood buffer
[NVML] GPU: 36%  |  MemCtl: 2%  |  VRAM: 13.90/15.00 GiB (92.7%)  |  Temp: 47 °C
[NVML] GPU: 100%  |  MemCtl: 19%  |  VRAM: 13.91/15.00 GiB (92.7%)  |  Temp: 48 °C
[NVML] GPU: 100%  |  MemCtl: 6%  |  VRAM: 13.91/15.00 GiB (92.7%)  |  Temp: 49 °C
[NVML] GPU: 100%  |  MemCtl: 7%  |  VRAM: 13.91/15.00 GiB (92.7%)  |  Temp: 50 °C
[NVML] GPU: 100%  |  MemCtl: 6%  |  VRAM: 13.91/15.00 GiB (92.7%)  |  Temp: 50 °C
[NVML] GPU: 100%  |  MemCtl: 6%  |  VRAM: 13.91/15.00 GiB (92.7%)  |  Temp: 50 °C
[NVML] GPU: 100%  |  MemCtl: 7%  |  VRAM: 13.91/15.00 GiB (92.7%)  |  Temp: 51 °C
[NVML] GPU: 100%  |  MemCtl: 7%  |  VRAM: 13.91/15.00 GiB (92.7%)  |  Temp: 51 °C
[NVML] GPU: 100%  |  MemCtl: 5%  |  VRAM: 13.91/15.